In [7]:
import polars as pl
import pysentiment2 as ps
import math
import os
import re
import urllib.parse

In [8]:
lm_dict = pl.read_csv(r'C:\Users\310\Desktop\Progects_Py\Parsim-sec\src\Analysis\Loughran-McDonald_MasterDictionary_1993-2021.csv')
lm_dict

Word,Seq_num,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Strong_Modal,Weak_Modal,Constraining,Syllables,Source
str,i64,i64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""AARDVARK""",1,354,1.5501e-8,1.4226e-8,0.000004,99,0,0,0,0,0,0,0,2,"""12of12inf"""
"""AARDVARKS""",2,3,1.3136e-10,8.6538e-12,9.2417e-9,1,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ABACI""",3,9,3.9409e-10,1.1697e-10,5.2905e-8,7,0,0,0,0,0,0,0,3,"""12of12inf"""
"""ABACK""",4,29,1.2698e-9,6.6547e-10,1.5951e-7,28,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ABACUS""",5,8570,3.7526e-7,3.8095e-7,0.000035,1108,0,0,0,0,0,0,0,3,"""12of12inf"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""ZYGOTE""",86529,50,2.1894e-9,8.7293e-10,1.8860e-7,35,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ZYGOTES""",86530,1,4.3788e-11,1.8095e-11,1.9324e-8,1,0,0,0,0,0,0,0,2,"""12of12inf"""
"""ZYGOTIC""",86531,0,0.0,0.0,0.0,0,0,0,0,0,0,0,0,3,"""12of12inf"""


In [24]:
df_dict = {}

for company_name in os.listdir(r"C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred"):
    
    file_path = os.path.join(r"C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred", company_name)

    match = re.search(r"\\([A-Z]+(?:\.[A-Z])?)_reports\.parquet", file_path)

    if match:
        ticker_symbol = match.group(1)
        print("Ticker Symbol:", ticker_symbol, '\n', file_path)
    else:
        print(f"Ticker symbol for {company_name} is not found")

    df_name = f"{ticker_symbol}"
    df = pl.read_parquet(file_path)

    df_dict[df_name] = df


Ticker Symbol: AEE 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AEE_reports.parquet
Ticker Symbol: AKAM 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AKAM_reports.parquet
Ticker Symbol: ALGN 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ALGN_reports.parquet
Ticker Symbol: AME 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AME_reports.parquet
Ticker Symbol: AMT 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AMT_reports.parquet
Ticker Symbol: AMZN 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AMZN_reports.parquet
Ticker Symbol: ANSS 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ANSS_reports.parquet
Ticker Symbol: ARE 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\ARE_reports.parquet
Ticker Symbol: AXON 
 C:\Users\310\Desktop\Progects_Py\prepared_data\full_snp_five_hundred\AXON_reports.parquet


In [4]:
#old version for dfs without 10-K reports, full text (downloaded, from div)
df_dict = {}

for company_name in COMPANY_NAME_LIST:

    company_name_dec = urllib.parse.unquote_plus(company_name)

    match = re.search(r'\((.*?)\)', company_name_dec)

    if not match:
        print(f"Ticker symbol for {company_name} is not found")

    reg_output = match.group(1).split('%')
    ticker_symbol = reg_output[0]

    print(ticker_symbol)

    df_name = f"{ticker_symbol}"

    file_path_partial = os.path.join(r"C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels", company_name)
    file_path = file_path_partial + '_reports.parquet'
    print(file_path)

    df = pl.read_parquet(file_path)

    df_dict[df_name] = df



AAPL
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\Apple%2520Inc.%2520(AAPL)%2520(CIK%25200000320193)_reports.parquet
MSFT
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\MICROSOFT%2520CORP%2520(MSFT)%2520(CIK%25200000789019)_reports.parquet
BIGC
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\BigCommerce%2520Holdings%252C%2520Inc.%2520(BIGC)%2520(CIK%25200001626450)_reports.parquet
ROKU
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\ROKU%252C%2520INC%2520(ROKU)%2520(CIK%25200001428439)_reports.parquet


JPM
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\JPMORGAN%2520CHASE%2520%2526%2520CO%2520(JPM%252C%2520AMJ%252C%2520AMJB%252C%2520JPM)%2520(CIK%25200000019617)_reports.parquet
V
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\VISA%2520INC.%2520(V)%2520(CIK%25200001403161)_reports.parquet
SQ
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\Block%252C%2520Inc.%2520(SQ%252C%2520BSQKZ)%2520(CIK%25200001512673)_reports.parquet
HOOD
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\Robinhood%2520Markets%252C%2520Inc.%2520(HOOD)%2520(CIK%25200001783879)_reports.parquet
JNJ
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\JOHNSON%252C%252C%2520JOHNSON%2520(JNJ)%2520(CIK%25200000200406)_reports.parquet
PFE
C:\Users\310\Desktop\Progects_Py\prepared_data\full_10Q_10K_without_tabels\PFIZER%2520INC%2520(PFE)%2520(CIK%25200000078003)_reports.parquet
MRNA
C:\Users\310\

In [25]:
total_num_of_rep = 0

for value in df_dict.values():
    total_num_of_rep += value.width
print(total_num_of_rep)

7807


In [26]:
total_num_of_words = 0

for value in df_dict.values():
    for col in value:
        total_num_of_words += col.count()

print(total_num_of_words)

208814303


In [27]:
#average doc length
average_word_count = total_num_of_words/total_num_of_rep
print(f'Average doc length in words: {round(average_word_count)}')
ln_average_word_count = math.log(average_word_count)

Average doc length in words: 26747


In [28]:
#ENSURE CASE CoMpAtAbIlItY
positive_words = lm_dict.filter(lm_dict["Positive"] > 0).to_series().str.to_lowercase()

negative_words = lm_dict.filter(lm_dict["Negative"] > 0).to_series().str.to_lowercase()
print(len(negative_words) + len(positive_words))



2692


In [30]:
#example series 
example_series = df_dict['ALGN'][df_dict['ALGN'].columns[1]]
example_df = df_dict['ALGN']
example_df


2019-02-28.parquet,2019-05-02.parquet,2019-08-01.parquet,2019-10-31.parquet,2020-02-28.parquet,2020-05-05.parquet,2020-07-31.parquet,2020-10-30.parquet,2021-02-26.parquet,2021-05-05.parquet,2021-08-04.parquet,2021-11-02.parquet,2022-02-25.parquet,2022-05-05.parquet,2022-08-04.parquet,2022-11-04.parquet,2023-02-27.parquet,2023-05-05.parquet,2023-08-04.parquet,2023-11-03.parquet
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED""","""UNITED"""
"""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…","""STATESSECURITI…"
"""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND""","""AND"""
"""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE""","""EXCHANGE"""
"""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…","""COMMISSIONWash…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
null,null,null,null,null,null,null,null,"""capacities""",null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,"""and""",null,null,null,null,null,null,null,null,null,null,null
null,null,null,null,null,null,null,null,"""on""",null,null,null,null,null,null,null,null,null,null,null


In [31]:
#here I have checked if there ara weird numder of words for reports
for company_name, df in df_dict.items():
    for col in df:
        if col.count() < 5000:
            print(f"{company_name}, {col.name}: {col.count()}")
        elif col.count() > 100000:
            print(f"{company_name}, {col.name}: {col.count()}")

BK, 2019-02-27.parquet: 4573
BK, 2020-02-27.parquet: 4046
BK, 2022-02-25.parquet: 4911
BX, 2019-03-01.parquet: 108861
BX, 2020-02-28.parquet: 101887
BX, 2021-02-26.parquet: 105295
BX, 2022-02-25.parquet: 104668
BX, 2023-02-24.parquet: 104516
CVS, 2023-02-08.parquet: 102008
C, 2021-02-26.parquet: 127501
C, 2022-02-28.parquet: 128374
C, 2023-02-27.parquet: 131435
DUK, 2021-02-25.parquet: 100697
EMR, 2020-02-05.parquet: 4067
EMR, 2021-02-03.parquet: 4342
ETR, 2019-02-26.parquet: 168747
ETR, 2020-02-21.parquet: 160811
ETR, 2021-02-26.parquet: 195291
ETR, 2022-02-25.parquet: 199557
ETR, 2023-02-24.parquet: 223235
EXC, 2019-02-08.parquet: 133056
EXC, 2021-02-24.parquet: 117172
EXC, 2022-02-25.parquet: 105179
FITB, 2021-02-26.parquet: 114400
FITB, 2022-02-25.parquet: 108102
FITB, 2023-02-24.parquet: 101084
GS, 2021-02-22.parquet: 106729
GS, 2022-02-25.parquet: 109072
GS, 2023-02-24.parquet: 131119
HIG, 2021-02-19.parquet: 110565
HIG, 2022-02-18.parquet: 105016
JPM, 2019-02-26.parquet: 104752


This is the func, that appled to a series and get sentiment words from it. As an output we get a df with first col - grouped words that appeaps in LOUGHRAN dictionary and has >0 value in "positive" or "negative" cols. Secon col contain count of corresponding word in this document. Third col contain boolean, indicatin if this word has positive sentiment (True) or negative (False)

In [33]:
def count_sentiment_w_in_doc(df_series):
        
        #grouped series that became a df, where each row is unique word and its count in document 
        num_of_w = df_series.value_counts().rename({"count": "count in doc"})
       
        
        #first col of this df (only words by itself)
        only_w = num_of_w[num_of_w.columns[0]]
        

        #Filterd df, there is only words that have positive or negative sentiment score in this document 
        pos_w = num_of_w.filter(only_w.is_in(positive_words)).with_columns(pl.lit(True).alias("positive"))
        neg_w = num_of_w.filter(only_w.is_in(negative_words)).with_columns(pl.lit(False).alias("positive"))
       
        #Staked for one df for further operations 
        df_stacked = neg_w.vstack(pos_w)

        return df_stacked
    
check = count_sentiment_w_in_doc(df_dict['MMM'][df_dict['MMM'].columns[1]])  
print(check)

shape: (215, 3)
┌────────────────────┬──────────────┬──────────┐
│ 2019-04-26.parquet ┆ count in doc ┆ positive │
│ ---                ┆ ---          ┆ ---      │
│ str                ┆ u32          ┆ bool     │
╞════════════════════╪══════════════╪══════════╡
│ lack               ┆ 2            ┆ false    │
│ violations         ┆ 3            ┆ false    │
│ inability          ┆ 1            ┆ false    │
│ retaliatory        ┆ 1            ┆ false    │
│ allegedly          ┆ 8            ┆ false    │
│ …                  ┆ …            ┆ …        │
│ gain               ┆ 9            ┆ true     │
│ improve            ┆ 8            ┆ true     │
│ greatest           ┆ 1            ┆ true     │
│ innovator          ┆ 1            ┆ true     │
│ innovation         ┆ 1            ┆ true     │
└────────────────────┴──────────────┴──────────┘


This is the func that counts a number of docs with at least one occurense of the input word. It takes hole dictionary whith dataframes and particular word as an input. Then checks how many cols in each df contain this word, sum up all resulst and return a single value for each word.

In [34]:
def in_all_doc_occurence(check_str, df_dict):
    count = 0
    for df in df_dict.values():
        count += df.select((pl.all() == check_str).sum() > 0).sum_horizontal()[0]
        
    return count

In [36]:
stringg = "a"
in_all_doc_occurence(stringg, df_dict)

7807

This is the func that takes a df from count_sentiment_w_in_doc() and for each word in this df applies in_all_doc_occurence(), meaning that it counts all docs with at least one occurence of this word. The result of applying this func for each word in df is a series of integers showing counts for correcdonding word. This series augmented af col named "count in all docs"

In [37]:
def total_occurence_count(df_series, df_dict):
   check = count_sentiment_w_in_doc(df_series).lazy()

   counts = check.with_columns(pl.col(check.columns[0]).map_elements(lambda x: in_all_doc_occurence(x, df_dict)).alias("count in all docs")).collect()
   return counts

In [38]:
res_df = total_occurence_count(example_series, df_dict)
print(res_df)

PanicException: python function failed KeyboardInterrupt: 

PanicException: python function failed KeyboardInterrupt: 

This is the func that takes as an argument resulting df, wich was obtained from total_occurence_count(). Recal that this df contains information about one document (one col in df). This func calculate weighted and raw scores using precomputed metrics. Returns a list with weighted score and raw score. Weights are calculated based on this formula: 

$$
w_{i,j} = 
\begin{cases} 
\left(\frac{1 + \log(tf_{i,j})}{1 + \log(a)}\right) \log\left(\frac{N}{df_i}\right) & \text{if } tf_{i,j} \geq 1 \\
0 & \text{otherwise} 
\end{cases}
$$


N represents the total number of 10-Ks in the sample, dfi the number of documents containing at least one occurrence of the i th word, tfi,j the raw count of the i th word in the j th document, and a the average word count in the document.

The formula is from:

Loughran T., McDonald B. When is a liability not a liability? Textual analysis, dictionaries, and 10‐Ks //The Journal of finance. – 2011. – Т. 66. – №. 1. – С. 35-65.

In [16]:
def compute_score(res_df):

    res_df.lazy()

    res = res_df.with_columns((((1 + pl.col(res_df.columns[1]).log())/(1 + ln_average_word_count))*((total_num_of_rep/pl.col(res_df.columns[3])).log())).alias("metrics"))

    w_pos_sen = res.filter(pl.col('positive') == True).select(pl.col(res.columns[4])).sum().item()
    w_neg_sen = res.filter(pl.col('positive') == False).select(pl.col(res.columns[4])).sum().item()

    w_score = w_pos_sen - w_neg_sen
    score = res.select(pl.col('positive')).sum().item()*2 - len(res)

    scores = [w_score, score]
    
    return scores



This is the function that takes polars series as an input, then concatinate all values in the single string. For this sitring, which is the initial report, this func calculates sentiment scores according to Harvard IV-4 and Loughran and McDonald Financial Sentiment Dictionaries. Perticulary, this function calculates polarity score which is defined by the following formula:

$$
Polarity = \frac{N_{\text{pos}} - N_{\text{neg}}}{N_{\text{pos}} + N_{\text{neg}}}
$$

In [17]:
hiv4 = ps.HIV4()
lm = ps.LM() 

def compute_pysentiment(series):

    doc_string = series.str.concat(" ", ignore_nulls=True)

    tokens_hiv4 = hiv4.tokenize(doc_string[0])
    tokens_lm = lm.tokenize(doc_string[0])

    score_hiv4 = hiv4.get_score(tokens_hiv4)
    score_lm = lm.get_score(tokens_lm)

  
    return score_lm["Polarity"], score_hiv4["Polarity"]

In [18]:
#Example of pysentiment2 working on example_series: first value is "Polsrity" score from Loughran-McDonald dictionary, and second one is from Harvard dictionary
lst = [1, 48.0303, 334, 'extended values starts from here:']

lst.extend(compute_pysentiment(example_series))

print(lst)

[1, 48.0303, 334, 'extended values starts from here:', -0.49927641063728184, 0.4112870482315698]


General loop that iterates over dfs and over it columns. It uses predefined finctions to calculate scores and save it as parquet files. Each parquet file named with company name is a dataframe with col names as field dates, first row is weighted scores and second row is raw scores. 

In [23]:

for company_name, df in df_dict.items():

    score_dict = {}

    for col in df:
        
        res_df = total_occurence_count(col, df_dict)
        scores = compute_score(res_df)

        scores.extend(compute_pysentiment(col))
        scores.append(col.count())

        score_dict[col.name] = scores
    
    df = pl.DataFrame(score_dict)
    
    print(f'Iteration for {company_name} is done successfully')

 # Determine the output directory and file name
    output_dir = os.path.join('data', 'full_10_Q_scores')
    os.makedirs(output_dir, exist_ok=True)

    file_name_new = f"{company_name}_full_doc_sen_score_df.parquet"
    full_path = os.path.join(output_dir, file_name_new)
    full_path = os.path.normpath(full_path)

    print(f"Attempting to write to: {full_path}")

    # Write the DataFrame to Parquet
    df.write_parquet(full_path)

    print(f"{full_path} created successfully")
        
        

Iteration for AAPL is done successfully
Attempting to write to: data\full_10_Q_scores\AAPL_full_doc_sen_score_df.parquet
data\full_10_Q_scores\AAPL_full_doc_sen_score_df.parquet created successfully
Iteration for MSFT is done successfully
Attempting to write to: data\full_10_Q_scores\MSFT_full_doc_sen_score_df.parquet
data\full_10_Q_scores\MSFT_full_doc_sen_score_df.parquet created successfully
Iteration for BIGC is done successfully
Attempting to write to: data\full_10_Q_scores\BIGC_full_doc_sen_score_df.parquet
data\full_10_Q_scores\BIGC_full_doc_sen_score_df.parquet created successfully
Iteration for ROKU is done successfully
Attempting to write to: data\full_10_Q_scores\ROKU_full_doc_sen_score_df.parquet
data\full_10_Q_scores\ROKU_full_doc_sen_score_df.parquet created successfully
Iteration for JPM is done successfully
Attempting to write to: data\full_10_Q_scores\JPM_full_doc_sen_score_df.parquet
data\full_10_Q_scores\JPM_full_doc_sen_score_df.parquet created successfully
Iteratio